<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4c4bc1e59d2e7551504006fef38bf0f748011f9711a67b5674d6ca221da0d20d
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-02 13:09:14
-------------------
qualified stocks: 92
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.17 L
Current:  1.39 C
-------------------
Today PnL: 1.17 L (0.85%)
Current PnL: -17.75 L (-11.94%)
CY Booked + Current PnL: -8.71 L (-5.86%)
-------------------
Total profit:  3.52 L
Total loss:  -21.27 L
-------------------
Total Booked + Current PnL: 18.27 L (15.02%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.48%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 79.41 L (56.93%)
Deployed:  1.22 C
Current:  1.39 C
CAGR/XIRR %: 8.65%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-16.60,64.0,H-LC,8.20,115552.0,11240.0,5408.0,0.36,10.78,4.68,15.96,9.0,2.08,0.84,29.49,X5K,ATH,METALS
53,MEDANTA,1486.00,13.45,59.0,H-MC,15.41,161901.0,35701.0,10475.0,-0.02,28.29,6.47,36.59,108.0,3.41,1.18,44.40,XY24,NTT,HEALTHCARE
26,DIXON,18940.15,-29.52,68.0,H-MC,6.66,140960.0,26024.0,10558.0,0.22,22.64,7.49,31.83,81.0,2.46,1.03,46.11,X40N,ATH,IT
82,TTKPRESTIG,770.00,107.26,71.0,M-SC,7.31,89250.0,-11527.0,11620.0,1.05,-11.44,13.02,0.09,245.0,-0.99,0.65,17.26,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-15.30,56.0,H-LC,8.68,169213.0,13036.0,13063.0,0.26,8.35,7.72,16.71,22.0,1.00,1.23,27.24,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,731.35,4.80,44.0,H-SC,11.08,124954.0,-343.0,59341.0,0.79,-0.27,47.49,47.09,141.0,-0.01,0.91,56.39,MH,ATH,POWER
32,HAVELLS,2069.16,0.20,63.0,H-MC,2.62,248212.0,481.0,74588.0,1.07,0.19,30.05,30.30,92.0,0.01,1.81,13.51,X40,ATH,ELECTRICAL
38,ICICIGI,2260.25,-21.04,34.0,H-MC,4.66,135657.0,-476.0,31608.0,-0.39,-0.35,23.30,22.86,91.0,-0.02,0.99,12.22,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,JIOFIN,387.00,-0.66,43.0,H-LC,14.70,219765.0,2359.0,51139.0,-0.03,1.09,23.27,24.61,48.0,0.05,1.60,56.50,XY24,BTT,FINANCE
33,HCLTECH,1943.91,1.51,44.0,H-LC,9.94,231387.0,-10537.0,73812.0,0.37,-4.36,31.90,26.15,8.0,-0.14,1.69,9.34,X40,ATH,IT
61,RELIANCE,1533.00,-14.04,45.0,H-LC,7.86,215062.0,4696.0,24087.0,1.82,2.23,11.20,13.68,37.0,0.19,1.57,19.12,XY25,NTT,REFINERIES
1,ABB,7934.00,-40.64,47.0,H-LC,13.62,247920.0,-13699.0,132910.0,0.73,-5.24,53.61,45.57,7.0,-0.10,1.81,5.49,AR,NTT,ELECTRICAL
45,ITC,452.00,-38.34,47.0,H-LC,3.23,198827.0,-1311.0,22209.0,0.18,-0.66,11.17,10.44,4.0,-0.06,1.45,4.87,X40,NTT,FMCG


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LTIM,7230.20,-4.62,58.0,H-LC,2.46,236700.0,-9451.0,88668.0,1.21,-3.84,37.46,32.18,16.0,-0.11,1.72,30.82,X200,ATH,IT
55,NESTLEIND,1377.00,-8.86,62.0,H-LC,2.87,280285.0,14859.0,41931.0,1.99,5.60,14.96,21.40,11.0,0.35,2.04,13.09,XY25,NTT,FMCG
45,ITC,452.00,-38.34,47.0,H-LC,3.23,198827.0,-1311.0,22209.0,0.18,-0.66,11.17,10.44,4.0,-0.06,1.45,4.87,X40,NTT,FMCG
20,CIPLA,1795.00,-18.75,60.0,H-LC,4.93,216666.0,12166.0,29250.0,-0.36,5.95,13.50,20.25,10.0,0.42,1.58,15.20,X40N,BTT,PHARMA
88,VBL,671.64,-11.76,48.0,H-LC,7.62,317852.0,2010.0,110644.0,0.58,0.64,34.81,35.67,5.0,0.02,2.31,14.58,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,NATIONALUM,247.44,-46.98,66.0,H-MC,0.99,106244.0,5740.0,24904.0,4.52,5.71,23.44,30.49,79.0,0.23,0.77,39.97,MH,ATH,METALS
81,TRIDENT,48.00,-8.84,51.0,M-SC,6.42,70938.0,-21316.0,47621.0,4.10,-23.11,67.13,28.51,224.0,-0.45,0.52,21.39,XR,NTT,TEXTILES
58,RAJESHEXPO,518.00,1658.84,46.0,L-SC,8.77,48002.0,-89175.0,89269.0,4.04,-65.01,185.97,0.07,267.0,-1.00,0.35,19.38,OX40N,NTT,JEWELLERY
24,DABUR,735.00,-2.76,65.0,H-MC,3.74,211848.0,14820.0,74804.0,3.73,7.52,35.31,45.49,102.0,0.20,1.54,21.54,XY24,BTT,FMCG
25,DEN,75.00,104.66,47.0,M-SC,3.38,57664.0,-26800.0,62335.0,3.03,-31.73,108.10,42.07,237.0,-0.43,0.42,22.46,AR,NTT,ENTERTAINMENT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,HONAUT,58357.33,-19.80,46.0,H-SC,3.47,113070.0,-14868.0,62008.0,-1.81,-11.62,54.84,36.84,143.0,-0.24,0.82,15.76,X40N,ATH,ELECTRICAL
40,INDIAMART,4850.92,-51.47,46.0,H-SC,12.53,135489.0,12153.0,121615.0,-1.68,9.85,89.76,108.45,119.0,0.10,0.99,34.41,AR,ATH,MISC
73,STARHEALTH,761.00,8.28,53.0,H-SC,2.77,161009.0,-39207.0,115234.0,-1.32,-19.58,71.57,37.97,144.0,-0.34,1.17,29.28,XY24,NTT,INSURANCE
52,MASFIN,398.61,-19.15,46.0,H-SC,13.15,91845.0,-6135.0,27737.0,-1.26,-6.26,30.20,22.05,152.0,-0.22,0.67,33.50,XR,ATH,FINANCE
27,DMART,5201.00,-8.17,65.0,H-LC,13.89,229256.0,44775.0,25585.0,-1.19,24.27,11.16,38.14,38.0,1.75,1.67,37.45,X40N,NTT,FMCG


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TTKPRESTIG,770.0,107.26,71.0,M-SC,7.31,89250.0,-11527.0,11620.0,1.05,-11.44,13.02,0.09,245.0,-0.99,0.65,17.26,OX40N,NTT,DURABLES
83,UJJIVANSFB,60.0,84.36,45.0,H-SC,14.11,116667.0,-25812.0,45337.0,-0.23,-18.12,38.86,13.70,163.0,-0.57,0.85,37.70,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.34,54.0,H-SC,3.67,218619.0,-51048.0,87382.0,0.53,-18.93,39.97,13.47,138.0,-0.58,1.59,10.57,XY24,NTT,PAINTS
41,INDIGOPNTS,1408.0,113.73,48.0,M-SC,2.79,140046.0,-34513.0,34549.0,2.86,-19.77,24.67,0.02,221.0,-1.00,1.02,21.82,OX40N,NTT,PAINTS
19,CERA,9475.0,-21.00,40.0,H-SC,3.17,112869.0,-30934.0,57676.0,-0.26,-21.51,51.10,18.60,149.0,-0.54,0.82,24.55,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,4998.00,-26.44,50.0,H-LC,15.05,292584.0,-53372.0,177218.0,0.90,-15.43,60.57,35.80,1.0,-0.30,2.13,3.65,X40,BTT,IT
43,INFY,2275.00,-17.44,52.0,H-LC,11.42,321119.0,8231.0,163450.0,0.55,2.63,50.90,54.87,3.0,0.05,2.34,9.45,X40,BTT,IT
45,ITC,452.00,-38.34,47.0,H-LC,3.23,198827.0,-1311.0,22209.0,0.18,-0.66,11.17,10.44,4.0,-0.06,1.45,4.87,X40,NTT,FMCG
88,VBL,671.64,-11.76,48.0,H-LC,7.62,317852.0,2010.0,110644.0,0.58,0.64,34.81,35.67,5.0,0.02,2.31,14.58,X40N,ATH,FMCG
1,ABB,7934.00,-40.64,47.0,H-LC,13.62,247920.0,-13699.0,132910.0,0.73,-5.24,53.61,45.57,7.0,-0.10,1.81,5.49,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,13.45,59.0,H-MC,15.41,161901.0,35701.0,10475.0,-0.02,28.29,6.47,36.59,108.0,3.41,1.18,44.40,XY24,NTT,HEALTHCARE
34,HEROMOTOCO,5949.07,0.33,77.0,H-MC,18.62,185762.0,34849.0,22459.0,1.21,23.09,12.09,37.97,93.0,1.55,1.35,54.24,AR,ATH,AUTO
26,DIXON,18940.15,-29.52,68.0,H-MC,6.66,140960.0,26024.0,10558.0,0.22,22.64,7.49,31.83,81.0,2.46,1.03,46.11,X40N,ATH,IT
27,DMART,5201.00,-8.17,65.0,H-LC,13.89,229256.0,44775.0,25585.0,-1.19,24.27,11.16,38.14,38.0,1.75,1.67,37.45,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,13.45,59.0,H-MC,15.41,161901.0,35701.0,10475.0,-0.02,28.29,6.47,36.59,108.0,3.41,1.18,44.40,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,12.69,65.0,H-MC,3.42,194490.0,29820.0,38139.0,-0.17,18.11,19.61,41.28,89.0,0.78,1.42,27.54,X40N,ATH,AC
89,VOLTAS,1918.49,2.42,69.0,H-MC,5.37,214950.0,23208.0,72825.0,1.49,12.10,33.88,50.08,99.0,0.32,1.57,19.72,XY25,ATH,AC
34,HEROMOTOCO,5949.07,0.33,77.0,H-MC,18.62,185762.0,34849.0,22459.0,1.21,23.09,12.09,37.97,93.0,1.55,1.35,54.24,AR,ATH,AUTO
32,HAVELLS,2069.16,0.20,63.0,H-MC,2.62,248212.0,481.0,74588.0,1.07,0.19,30.05,30.30,92.0,0.01,1.81,13.51,X40,ATH,ELECTRICAL


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,0.33,77.0,H-MC,18.62,185762.0,34849.0,22459.0,1.21,23.09,12.09,37.97,93.0,1.55,1.35,54.24,AR,ATH,AUTO
54,NATIONALUM,247.44,-46.98,66.0,H-MC,0.99,106244.0,5740.0,24904.0,4.52,5.71,23.44,30.49,79.0,0.23,0.77,39.97,MH,ATH,METALS
90,WHIRLPOOL,2270.00,-40.79,56.0,M-SC,5.16,100102.0,8604.0,70151.0,1.37,9.40,70.08,86.07,223.0,0.12,0.73,45.37,XR,NTT,DURABLES
40,INDIAMART,4850.92,-51.47,46.0,H-SC,12.53,135489.0,12153.0,121615.0,-1.68,9.85,89.76,108.45,119.0,0.10,0.99,34.41,AR,ATH,MISC
91,WIPRO,420.00,-11.07,55.0,M-LC,7.12,156445.0,5500.0,103958.0,0.79,3.64,66.45,72.51,53.0,0.05,1.14,9.89,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-3.83,45.0,H-SC,16.34,89483.0,-11480.0,106011.0,2.22,-11.37,118.47,93.63,148.0,-0.11,0.65,33.28,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,68.0,L-SC,31.33,92448.0,-21101.0,61154.0,1.97,-18.58,66.15,35.27,268.0,-0.35,0.67,126.27,XR,NTT,HOTELS
52,MASFIN,398.61,-19.15,46.0,H-SC,13.15,91845.0,-6135.0,27737.0,-1.26,-6.26,30.20,22.05,152.0,-0.22,0.67,33.50,XR,ATH,FINANCE
35,HINDALCO,756.01,-16.60,64.0,H-LC,8.20,115552.0,11240.0,5408.0,0.36,10.78,4.68,15.96,9.0,2.08,0.84,29.49,X5K,ATH,METALS
83,UJJIVANSFB,60.00,84.36,45.0,H-SC,14.11,116667.0,-25812.0,45337.0,-0.23,-18.12,38.86,13.70,163.0,-0.57,0.85,37.70,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,68.0,L-SC,31.33,92448.0,-21101.0,61154.0,1.97,-18.58,66.15,35.27,268.0,-0.35,0.67,126.27,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,0.33,77.0,H-MC,18.62,185762.0,34849.0,22459.0,1.21,23.09,12.09,37.97,93.0,1.55,1.35,54.24,AR,ATH,AUTO
14,BLUESTARCO,2326.38,12.69,65.0,H-MC,3.42,194490.0,29820.0,38139.0,-0.17,18.11,19.61,41.28,89.0,0.78,1.42,27.54,X40N,ATH,AC
27,DMART,5201.00,-8.17,65.0,H-LC,13.89,229256.0,44775.0,25585.0,-1.19,24.27,11.16,38.14,38.0,1.75,1.67,37.45,X40N,NTT,FMCG
60,RELAXO,1176.00,-33.20,58.0,H-SC,7.20,61990.0,-53548.0,88534.0,-1.10,-46.35,142.82,30.28,136.0,-0.60,0.45,21.87,X40N,NTT,FOOTWEAR


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.34
1,25,42.59
2,50,72.19


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
LC    37.62
SC    36.37
MC    26.02
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.72
X40      15.23
X40N     13.93
XY25     11.23
AR        9.18
XR        8.81
OX40N     7.19
X5K       2.25
X200      1.72
MH        1.68
SR        1.07
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.60
H-MC    23.19
H-SC    21.80
M-SC    13.01
M-LC     6.99
M-MC     2.52
L-SC     1.56
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.96,0.64,31.72
IT,13.15,-14.42,71.17
BANKS,7.15,-16.07,65.35
FINANCE,7.09,-24.15,69.93
PAINTS,5.85,-14.91,40.63
MISC,5.74,-16.78,83.73
ELECTRICAL,5.60,-7.16,66.22
AUTO,4.49,-9.48,57.11
AC,3.70,4.51,27.83


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2829735.0,23
AR,1220757.0,10
XR,1083143.0,13
X40,877900.0,10
X40N,520139.0,11
OX40N,470596.0,10
XY25,439346.0,8
SR,196912.0,2
X5K,130108.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2784160.0,27
M-SC,1459271.0,17
H-LC,1405728.0,18
H-MC,1193736.0,18
M-LC,418740.0,5
M-MC,331178.0,2
L-SC,246711.0,3
L-MC,59461.0,1
L-LC,42564.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,939433.0,6
M-SC,XY24,859989.0,7
H-SC,AR,666195.0,5
H-LC,X40,585727.0,5
H-SC,XR,526539.0,6
H-MC,XY24,472695.0,5
H-LC,AR,372230.0,2
M-MC,XY24,331178.0,2
H-LC,X40N,219179.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
